In [1]:
import torch
import matplotlib.pyplot as plt

## 3. Mutual Information 정의
- **목표:** MI의 수학적 정의와 직관 이해
- **핵심 개념:**
```
Joint / Marginal Distribution 이해
Mutual Information
```

### Joint / Marginal Distribution 이해
Joint Distribution
-  결합 분포란 확률 변수가 여러 개일 때 이들을 함께 고려하는 확률 분포이다. 
- $P(X=x \, and \, Y=y) \\ 
= P(Y=y|X=x)P(X=x) \\
= P(Y=y|X=x)P(Y=y) \\$
- 확률이기 때문에 $\\\sum_x\sum_yP(X=x \, and \, Y=y) = 1$
- $P(X=x \, and \, Y=y) = P(X=x) \cdot P(Y=y)$이면 X, Y는 독립
- $X$와 $Y$가 동시에 어떤 값을 가지는지를 나타냄

Marginal Distribution
- 주변 분포 (周邊分布) 란 그 부분 집합에 속한 확률 변수들의 확률 분포를 뜻한다. 이는 다른 확률 변수의 값을 무시한 부분 집합 속의 확률 변수의 분포를 알 수 있게 해준다. 이는 조건부 확률과 대비되는 부분이다.
- $f_X(x) = \sum^{\inf}_{y=0}f(x,y)dy$
- $f_Y(y) = \sum^{\inf}_{x=0}f(x,y)dx$
- $X$만, $Y$만 볼 때 확률
- 확률변수가 서로 독립일 때, 결합분포는 각 확률변수의 주변분포의 곱으로 표현된다.

아래 표를 통해 실습해보자
||Y=1|Y=2|Y=3|P(X=x)|
|---|---|---|---|---|
|X=0|0.2|0.175|0.125|0.5
|X=1|0.2|0.175|0.125|0.5
|P(Y=y)|0.4|0.35|0.25|1


In [5]:
prob = torch.tensor([[0.2	,0.175	,0.125],
                     [0.2	,0.175	,0.125]])

In [8]:
P_X = prob.sum(dim=1)
print("Marginal P(X):", P_X)

P_Y = prob.sum(dim=0)
print("Marginal P(Y):", P_Y)

P_X_Y = prob.sum()
print("P(X, Y):", P_X_Y)

Marginal P(X): tensor([0.5000, 0.5000])
Marginal P(Y): tensor([0.4000, 0.3500, 0.2500])
P(X, Y): tensor(1.)


In [12]:
conditional_P_X_Y = P_X_Y / P_Y
print("P(X|Y):", conditional_P_X_Y)

conditional_P_Y_X = P_X_Y / P_X
print("P(X|Y):", conditional_P_Y_X)

P(X|Y): tensor([2.5000, 2.8571, 4.0000])
P(X|Y): tensor([2., 2.])


### Mutual Information

두 개의 무작위 변수의 상호정보(또는 상호의존정보, Mutual information , MI)는 두 변수 사이의 <b>상호 의존성</b>을 측정 한 것이다. 보다 구체적으로 말하면 , 다른 무작위 변수를 통해 하나의 무작위 변수에 대해 얻어진 "정보량"을 정량화(계량화) 한다.    
상호 정보의 개념은 무작위 변수의 엔트로피의 개념과 복잡하게 연관되어 있으며, 무작위 변수의 "정보량"을 정의하는 정보 이론의 기본 개념이다.
- conditional entropy 관점
    - $I(X;Y)=H(X)−H(X|Y)$
- KL Divergence 관점
    - $I(X;Y)=D_{KL}​(P(X,Y)||P(X)P(Y))$
    - $I(X;Y)=\sum_{y\in Y}\sum_{x\in X}P(x,y)log_2{P(x,y) \over P(x)P(y)}$

In [44]:
import torch

def mutual_information(Pxy: torch.Tensor):
    Px = Pxy.sum(dim=1)  # [2]
    Py = Pxy.sum(dim=0)  # [3]

    # 그냥 Px * Py하게 되면 차원이 안맞아서 에러가 발생함
    # (2, 1) 과 (1, 3)으로 reshape
    Px_expand = Px[:, None]  # shape = [2, 1]
    Py_expand = Py[None, :]  # shape = [1, 3]

    # log2(0) 방지용 mask
    mask = Pxy > 0

    MI = torch.sum(Pxy[mask] * torch.log2(Pxy[mask] / (Px_expand * Py_expand)[mask]))
    return MI

In [47]:
print("Independent case I(X;Y) =", mutual_information(prob).item(), "bits")
Pxy = torch.tensor([[0.3, 0.05, 0.05, 0.05],
                    [0.05, 0.1, 0.4, 0.]])

print("Dependent case I(X;Y) =", mutual_information(Pxy).item(), "bits")

Independent case I(X;Y) = 0.0 bits
Dependent case I(X;Y) = 0.4214782416820526 bits


Thank you GPT!   

MI를 “극대화”한다는 말의 의미

X와 Y가 독립하지 않게 만드는 것,
즉 서로 강하게 연관되도록 만드는 것

직관적 비유
I(X;Y)=0 → X와 Y는 완전 독립. Y를 알아도 X에 대한 정보가 없음.
I(X;Y)이 클수록 → Y를 보면 X를 거의 확실히 예측할 수 있음.

따라서 MI를 극대화한다는 것은
“Y가 X를 최대한 잘 설명하도록 표현을 학습한다”는 의미입니다.

| 분야                                         | X            | Y                       | MI 극대화의 의미                           |
| ------------------------------------------ | ------------ | ----------------------- | ------------------------------------ |
| **Representation Learning**                | 원본 데이터       | 잠재 표현 (feature)         | 잠재 표현이 입력의 정보를 최대한 보존하도록 학습          |
| **Contrastive Learning (InfoNCE, SimCLR)** | anchor 이미지   | augment된 이미지            | 두 뷰가 같은 의미적 정보를 공유하도록 학습             |
| **Unsupervised Segmentation (IIC 등)**      | 원본 픽셀 / 클러스터 | augment된 이미지의 픽셀 / 클러스터 | 서로 대응하는 픽셀이 동일한 의미의 cluster에 속하도록 학습 |
| **Autoencoder / VAE 계열**                   | latent z     | input x                 | z가 x의 정보를 충분히 보존하도록 학습               |

MI 극대화의 핵심 목표

정보 보존 (Information Preservation)   
: Y(embedding, feature)가 X의 의미를 최대한 잃지 않게 함.

의미적 일관성 (Semantic Consistency)   
: 서로 다른 변환(augmentation, noise)이 있더라도 같은 의미를 공유하도록.

독립적 representation 제거 (redundancy reduction)   
: MI를 최대화하면서 동시에 feature 간 중복 정보는 최소화.